In [20]:
import sqlite3
import requests as req
import datetime
import os
import pyperclip
import webbrowser as wb
import pyautogui as gui
import time
import glob

In [43]:
#os.chdir('./')
#os.curdir

'.'

In [2]:
# DB 생성
conn = sqlite3.connect('Airport.db')
cur = conn.cursor()

In [36]:
# 테이블 생성
try:
    cur.execute('create table 항공정보(항공사,편명,출발지,계획,예상,도착,구분,현황,기준년월)')
except:
    pass
cur.execute('delete from 항공정보')

In [4]:
# 일자 생성 함수
class AirportalCrawler:
    def __init__(self, url, start, end):
        self.url = url
        self.start = start
        self.end = end
        
    def genDates(self):
        res = []
        delta = datetime.timedelta(days=1)
        date = start-delta
        for _ in range((end-start).days + 1):
            date = date+delta
            res.append(str(date))
        res = [dt.replace('-','') for dt in res]
        return res
    
    def parse(self, date):
        html = req.get(self.url, params={'current_date':str(date)}).text
        return html

In [5]:
url = 'http://www.airportal.go.kr/servlet/aips.life.airinfo.RbHanCTL'
start = datetime.date(2017,1,1)
end = datetime.date(2017,12,31)
ac = AirportalCrawler(url, start, end)
dates = ac.genDates()

In [6]:
for date in dates:
    html = ac.parse(date)
    n = len('<!-------- 게시판리스트시작 /------->')
    lb = html.find('<!-------- 게시판리스트시작 /------->')
    ub = html.find('<!-------- 게시판리스트끝 /------->')
    html = html[(lb+n):ub]
    html = html.replace('<TD with="4" align="center">','')
    html = html.replace('<td height="1" colspan="19" bgcolor="E5E6E6">','')
    html = html.replace('&nbsp;','')
    with open(date+'.html' , 'w') as f:
        f.write(html)

In [39]:
sql = 'insert into 항공정보 values (?,?,?,?,?,?,?,?,?)'
paths = glob.glob('./*.html')
for path in paths:
    date = path[2:10]
    wb.open(date+'.html')
    time.sleep(0.5)
    gui.hotkey('ctrl','a')
    gui.hotkey('ctrl','c')
    gui.hotkey('ctrl','w')
    data = pyperclip.paste()
    
    res = []
    for rows in data.split('\r\n'):
        row = rows.split('\t')
        row.append(date)
        res.append(row)
    res = res[:-1]
    cur.executemany(sql, res)

In [40]:
sql = 'select * from 항공정보'
cur.execute(sql)
rows = cur.fetchall()
rows

[('에티하드',
  'EY873',
  'AUH(아부다비)',
  '00:05',
  '00:05',
  '00:11',
  '여객',
  '출발',
  '20160101'),
 ('카타르항공',
  'QR859',
  'DOH(도하)',
  '00:05',
  '00:05',
  '00:30',
  '여객',
  '출발',
  '20160101'),
 ('싱가폴항공',
  'SQ603',
  'SIN(싱가포르)',
  '00:10',
  '00:10',
  '00:15',
  '여객',
  '출발',
  '20160101'),
 ('에미레이트항공',
  'EK323',
  'DXB(두바이)',
  '00:10',
  '23:50',
  '00:20',
  '여객',
  '출발',
  '20160101'),
 ('터키항공',
  'TK091',
  'IST(이스탄불)',
  '00:20',
  '00:05',
  '00:55',
  '여객',
  '출발',
  '20160101'),
 ('네덜란드항공',
  'KL856',
  'AMS(암스테르담)',
  '00:55',
  '00:55',
  '01:35',
  '여객',
  '출발',
  '20160101'),
 ('중국남방항공',
  'CZ316',
  'PEK(베이징)',
  '01:00',
  '01:00',
  '01:30',
  '여객',
  '출발',
  '20160101'),
 ('아시아나항공',
  'OZ987',
  'PVG(푸동)',
  '01:05',
  '00:51',
  '00:50',
  '화물',
  '출발',
  '20160101'),
 ('대한항공',
  'KE249',
  'JFK(뉴욕)',
  '02:30',
  '02:30',
  '02:57',
  '화물',
  '출발',
  '20160101'),
 ('홍콩익스프레스',
  'UO615',
  'HKG(홍콩)',
  '04:10',
  '04:10',
  '04:07',
  '여객',
  '출발',
  '2016010